In [2]:
import json

file_path = "../../../output_final.json"
with open(file_path, 'r') as json_file:
    data = json.load(json_file)

list(data.keys())

FileNotFoundError: [Errno 2] No such file or directory: '../../../output_final.json'

In [51]:
products = []
b = [] #to check duplicates
failed = []
base_url = 'https://www.fairprice.com.sg/product/'

In [52]:
dups = 0

def get_offer_qty(item):
    try:
        rule = item['offers'][0]['rule']
        for _, v in rule['buy'].items(): #strong assumption only one kind of offers avail at one time
            return v['q'], item['offers'][0]['price'],item['offers'][0]['description'] 
    except:
        return None, None, None
    

for cat, all_items in data.items():
    for item in all_items:
        product = {}
        present = False
        barcodes = item['barcodes']
        
        #check for duplicates
        if barcodes:
            for barcode in barcodes:
                if barcode in b:
                    #PRODUCT already within dict, add additional category if needed
                    present = True
                else:
                    b.append(barcode)
                    dups+=1
        
        if present:
            continue
            
        try:
            product['product_url'] = base_url + item['slug']
            product['market_id'] = 1
            product['barcodes'] = barcodes
            product['brand'] = item['brand']['clientId']
            product['price'] = item['final_price']
            product['availability'] = item['has_stock']
            product['name'] = item['name']
            
            try:
                product['image'] = item['images'][0]
            except:
                product['image'] = None
                
            try:
                product['quantity'] = item['metaData']['DisplayUnit']
            except:
                product['quantity'] = None
            product['offer_qty'],product['offer_price'],product['offer_desc'] = get_offer_qty(item)
       
            products.append(product)
        except Exception as e:
            failed.append([item, e])

In [53]:
len(failed)

0

In [54]:
len(products)

49835

In [25]:
failed[0][1]

TypeError("'NoneType' object is not subscriptable")

In [33]:
dups

52447

In [48]:
products[1]

{'url': 'https://www.fairprice.com.sg/product/coles-tuna-chunks-in-olive-oil-blend-95g-13213962',
 'market_id': 1,
 'barcodes': ['9300601173402', '9310645095196'],
 'brand': 'COLES',
 'price': 1.98,
 'availability': False,
 'name': 'Coles Tuna Chunks in Olive Oil Blend',
 'image': 'https://media.nedigital.sg/fairprice/fpol/media/images/product/XL/13213962_XL1_20240102.jpg',
 'quantity': '95g',
 'offer_qty': 1,
 'offer_price': 1.79,
 'offer_desc': 'Buy 1 Coles Tuna Chunks in Olive Oil Blend @ $1.79'}

In [55]:
import json

rand = {}
rand['all'] = products

file_path = "ntuc_cleaned.json"
with open(file_path, 'w') as json_file:
    json.dump(rand, json_file)